In [112]:
import numpy as np
import random
import pandas as pd
from itertools import chain

In [113]:
def gerar_array_replicavel(seed: int, tamanho: int) -> list[int]:
    random.seed(seed)
    vetor = list(range(tamanho))  
    random.shuffle(vetor)
    return vetor

In [ ]:
def calcular_aptidao_qap(solucao, matriz_fluxo, matriz_distancia):
    solucao_flat = []
    for x in solucao:
        if isinstance(x, (list, np.ndarray)):  
            solucao_flat.extend(int(v) for v in x)
        else:
            solucao_flat.append(int(x))
    
    n = len(solucao_flat)
    custo = 0
    for i in range(n):
        for j in range(n):
            custo += matriz_fluxo[i][j] * matriz_distancia[solucao_flat[i]][solucao_flat[j]]
    return custo


In [115]:
def inicializar_feromonio(n, valor_inicial=1.0):
    return np.full((n, n), valor_inicial)

In [116]:
def construir_solucao_qap(feromonio, alpha, beta, heuristica):
    n = feromonio.shape[0]
    nao_visitados = list(range(n))
    solucao = []

    while nao_visitados:
        if not solucao:
            atual = random.choice(nao_visitados)
        else:
            ultima = solucao[-1]
            pesos = []
            for prox in nao_visitados:
                tau = feromonio[ultima][prox] ** alpha
                eta = heuristica[ultima][prox] ** beta
                pesos.append(tau * eta)
            pesos = np.array(pesos)
            pesos /= pesos.sum()
            atual = random.choices(nao_visitados, weights=pesos)[0]

        solucao.append(atual)
        nao_visitados.remove(atual)

    return solucao


In [ ]:
def busca_local(solucao, fluxo, distancia):
    melhor = solucao.copy()
    melhor_custo = calcular_aptidao_qap(melhor, fluxo, distancia)
    melhorou = True
    while melhorou:
        melhorou = False
        for i in range(len(solucao) - 1):
            for j in range(i + 1, len(solucao)):
                nova = melhor.copy()
                nova[i], nova[j] = nova[j], nova[i]
                custo_novo = calcular_aptidao_qap(nova, fluxo, distancia)
                if custo_novo < melhor_custo:
                    melhor, melhor_custo = nova, custo_novo
                    melhorou = True
    return melhor

In [118]:
def atualizar_feromonio(feromonio, solucoes, custos, rho, Q):
    feromonio *= (1 - rho)
    for solucao, custo in zip(solucoes, custos):
        deposito = Q / custo
        for i in range(len(solucao) - 1):
            feromonio[solucao[i]][solucao[i+1]] += deposito
        feromonio[solucao[-1]][solucao[0]] += deposito
    return feromonio

In [ ]:
def aco_qap(fluxo, distancia, n_formigas=10, alpha=1, beta=2, rho=0.1, Q=1, iter_max=100):
    n = len(fluxo)
    heuristica = 1 / (np.array(distancia) + 1e-10)  
    feromonio = inicializar_feromonio(n)

    melhor_solucao = None
    melhor_custo = float('inf')

    for _ in range(iter_max):
        solucoes = []
        custos = []
        for _ in range(n_formigas):
            s = construir_solucao_qap(feromonio, alpha, beta, heuristica)
            s = busca_local(s, fluxo, distancia)  
            custo = calcular_aptidao_qap(s, fluxo, distancia)
            solucoes.append(s)
            custos.append(custo)
            if custo < melhor_custo:
                melhor_solucao, melhor_custo = s, custo

        feromonio = atualizar_feromonio(feromonio, solucoes, custos, rho, Q)

    return melhor_solucao, melhor_custo

In [120]:
def ler_qap_com_n(caminho: str):
    with open(caminho, "r") as f:
        dados = list(map(int, f.read().split()))
        
    n = dados[0]
    valores = dados[1:]  

    total_esperado = 2 * n * n
    if len(valores) != total_esperado:
        raise ValueError(f"Esperado {total_esperado} valores, mas encontrado {len(valores)}.")

    flow_flat = valores[:n * n]
    dist_flat = valores[n * n:]

    flow_df = pd.DataFrame([flow_flat[i * n:(i + 1) * n] for i in range(n)])
    dist_df = pd.DataFrame([dist_flat[i * n:(i + 1) * n] for i in range(n)])

    return n, flow_df, dist_df

In [121]:
n, flow_df, dist_df = ler_qap_com_n("Chr12a.txt")
flow = flow_df.values.tolist()
dist = dist_df.values.tolist()
matriz_fluxo = np.array(flow)
matriz_distancia = np.array(dist)

In [122]:
solucao = aco_qap(matriz_fluxo, matriz_distancia, n_formigas=10, alpha=1, beta=2, rho=0.1, Q=1, iter_max=100)

In [124]:
calcular_aptidao_qap(solucao=solucao[0], matriz_fluxo=flow, matriz_distancia=dist)

9552